# Modelo de Jia et al.

$$ \begin{align*}
\frac{dS}{dt} & = - \beta S (I + \theta A) - p S + \lambda Q
              &&= - \beta S I  -  \phi S A - p S + \lambda Q \\
\frac{dQ}{dt} & = p S - \lambda Q \\
\frac{dE}{dt} & =   \beta S (I + \theta A) - \sigma E
              &&=   \beta S I  +  \phi S A - \sigma E \\
\frac{dA}{dt} & = \sigma (1 - \rho) E - \epsilon_A A - \gamma_A A
              &&= (\sigma - \zeta) E  - \epsilon_A A - \gamma_A A \\
\frac{dI}{dt} & = \sigma \rho E - \epsilon_I I - \gamma_I I - d_I I
              &&=       \zeta E - \epsilon_I I - \gamma_I I - d_I I \\
\frac{dD}{dt} & = \epsilon_A A + \epsilon_I I - \gamma_D D - d_D D \\
\frac{dR}{dt} & = \gamma_A A + \gamma_I I + \gamma_D D
\end{align*} $$

## Modelo em python

Separado em uma biblioteca dedicada, `jia.py`.

In [33]:
6718903*2

13437806

In [38]:
2e5/6718903*2 *100

5.9533528017892205

In [41]:
(2.017e7-2.0002e7)/2.017e7 *100

0.8329201784828953

In [1]:
import git
import sys

root_dir = git.Repo('.', search_parent_directories=True).working_tree_dir
sys.path.insert(1, root_dir)

In [2]:
import modcovid as mc
from modcovid import scripts, settings

In [3]:
from scipy.integrate import solve_ivp
from jia import multi_regime, F, new_diagn, new_deaths

In [4]:
cidades = ['Niteroi', 'Campos Dos Goytacazes', 'Teresopolis', 'Duque De Caxias', 'Angra Dos Reis', 'Volta Redonda', 'Cabo Frio']
dados = {k: {} for k in cidades}

# Niterói

## Dados iniciais para Niterói

Usamos uma taxa de subnotificação de 8x, e uma taxa de exposição de 7x a população diagnosticada,
o que aproxima de forma bastante razoável a curva de diagnósticos e de óbitos.

Também é necessário estimar a taxa de assintomáticos nesta população,
que é maior do que para a doença já que a prioridade dos testes
é para aqueles que apresentaram sintomas mais graves.

In [5]:
cidade_atual = 'Niteroi'

In [6]:
# Taxa de subnotificação
taxa = {'taxa_subn': 8, 'prop_assint':0.80}
dados[cidade_atual].update(taxa)

In [7]:
# Dados para a cidade de Niterói
# IBGE 2019 (estimativa)

pop = 6718903

diagnosticados = 489
recuperados = 16
infectados = diagnosticados*(dados[cidade_atual]['taxa_subn']-1)*(1 - dados[cidade_atual]['prop_assint'])
assintomaticos = diagnosticados*(dados[cidade_atual]['taxa_subn']-1)*dados[cidade_atual]['prop_assint']
expostos = diagnosticados*7
quarentena = 0.6 * pop

sucetiveis = pop - (quarentena + expostos + assintomaticos + infectados + diagnosticados + recuperados)

dados[cidade_atual].update({'pop': pop, 'diagnosticados': diagnosticados, 'recuperados': recuperados, 'infectados': infectados, 'assintomaticos': assintomaticos, 'expostos': expostos, 'quarentena': quarentena})

In [8]:
# Idx       :         0,    1,   2,      3,     4,    5,     6,      7,    8,      9,     10,     11,     12
# Parâmetros:      beta,  phi,   p, lambda, sigma, zeta,  epsA, gammaA, epsI, gammaI, deathI, gammaD, deathD
params_name = ['beta', 'phi', 'p', 'lambda','sigma', 'zeta', 'epsA', 'gammaA', 'epsI', 'gammaI', 'deathI', 'gammaD', 'deathD']
params = (1e-7, 1/10, 1/4,    1/6,   1/7, 0.94, 1/100, 0.0996, 1/30, 0.0766, 0.0026, 0.0843, 0.0017)

In [9]:
dados[cidade_atual].update({'params': {k: v for k,v in zip(params_name, params)}})

In [10]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import datetime as dt
import pandas as pd

In [11]:
df2 = pd.read_csv('BICOVIDREAIS_19maio.csv')

In [12]:
df = mc.scripts.set_df('estado_rj', {'df_break': False})

In [13]:
df.dropna(subset = ['Municipio'], inplace = True)

In [14]:
df_work = df[df['Municipio'].str.contains('|'.join(cidades))]

In [15]:
type(df2['dt_sintoma'][0])

str

In [31]:
df2['dt_coleta_dt_notif'].unique()

array(['2020-05-16', '2020-04-21', '2020-04-06', '2020-04-20',
       '2020-04-15', '2020-05-06', '2020-05-08', '2020-03-24',
       '2020-04-19', '2020-04-22', '2020-04-08', '2020-04-30',
       '2020-05-11', '2020-04-14', '2020-05-12', '2020-05-14',
       '2020-04-29', '2020-05-13', '2020-05-04', '2020-03-30',
       '2020-04-27', '2020-05-15', '2020-04-17', '2020-04-16',
       '2020-04-24', '2020-05-05', '2020-04-18', '2020-03-27',
       '2020-04-09', '2020-05-07', '2020-04-28', '2020-04-13',
       '2020-05-01', '2020-03-31', '2020-04-04', '2020-05-03',
       '2020-04-12', '2020-04-11', '2020-05-19', '2020-04-07',
       '2020-04-02', '2020-05-09', '2020-04-23', '2020-04-05',
       '2020-04-25', '2020-03-28', '2020-05-10', '2020-04-10', nan,
       '2020-05-02', '2020-03-20', '2020-04-26', '2020-03-17',
       '2020-03-29', '2020-04-03', '2020-04-01', '2020-05-18',
       '2020-03-23', '2020-05-17', '2020-03-25', '2020-03-18',
       '2020-03-22', '2020-03-19', '2020-03-09', '

In [32]:
df2[df2['dt_coleta_dt_notif'] == '2020-01-09']

,sexo,idade,municipio_res,uf,dt_sintoma,dt_coleta_dt_notif,classificacao,evolucao,dt_obito,comorbidades,dias
16000,F,35.0,TERESOPOLIS,RJ,2020-01-01,2020-01-09,CONFIRMADO,NaN,NaN,NaN,139
16927,F,38.0,DUQUE DE CAXIAS,RJ,2020-01-04,2020-01-09,CONFIRMADO,NaN,NaN,NaN,136


In [17]:
df_ts = mc.scripts.get_timeseries(df, fonte = 'estado_rj')

In [18]:
df_ts

,Data,Japeri,Niteroi,Belford Roxo,Paraty,Duque De Caxias,Teresopolis,Nova Iguacu,Paty Do Alferes,Silva Jardim,...,Macuco,Cantagalo,Engenheiro Paulo De Frontin,Sao Jose De Uba,Cambuci,Comendador Levy Gasparian,Varre-Sai,Laje Do Muriae,Cardoso Moreira,Em Investigacao
0,2020-01-04,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,2020-01-05,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,2020-01-08,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,2020-01-09,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,2020-01-10,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2020-05-15,2.0,18.0,7.0,2.0,14.0,4.0,18.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77,2020-05-16,0.0,5.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103,2020-05-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81,2020-05-18,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
df_ts['Data'].head(20)

0     2020-01-04
39    2020-01-05
78    2020-01-08
82    2020-01-09
40    2020-01-10
83    2020-01-13
88    2020-01-16
90    2020-01-27
91    2020-01-29
85    2020-02-03
93    2020-02-04
97    2020-02-06
94    2020-02-09
98    2020-02-10
99    2020-02-11
41    2020-02-13
42    2020-02-17
95    2020-02-18
89    2020-02-19
101   2020-02-20
Name: Data, dtype: datetime64[ns]